# PeleeNet test
- 自宅PC tfgpu_py36環境
- C:\Users\shingo\Git\PeleeNet-Keras

- https://qiita.com/koshian2/items/187e240f478504079e7a

### PeleeNetと呼ばれるDenseNetベースの軽量化なモデル
### 係数が2.8Mと少ないながら、ImageNetでMobileNetV2以上の精度を出す
### ImageNetでPeleeNetがTop1:72.6%、MobileNetV2がTop1:72.0%
### PeleeNetでの高速化のポイントは、1x1畳み込みを非常に多く使って理論的な計算量を減らしている
- StemBlockは入力のダウンサンプリング（入力層のサイズを小さくする）用のブロック
    - 論文内のImageNetの実験でも、StemBlockの有無でTop1精度が75.8％→76.8％と1％向上
    
    
- DenseBlockとはDenseNetの「知識の積み重ね、集約」といったConcatで少しずつチャンネル数が増えていく構造のブロック


- Transition Layerとは全体に1x1Convを掛けてダウンサンプリングするレイヤー

In [1]:
import keras
print('keras.__version__:', keras.__version__)
import tensorflow as tf
print('tf.__version__:', tf.__version__)

import pelee_net_keras

Using TensorFlow backend.


keras.__version__: 2.1.5
tf.__version__: 1.8.0


- Stem blockの有無は、入力解像度・出力解像度が変わります。
- Stem blockありが本来のPeleeNetの構造です。なしの場合は、いきなりDenseLayerに入ります。


- Stem blockあり→入力 224 x 224 x 3：出力 7 x 7 x 704
- Stem blockなし→入力 32 x 32 x 3 : 出力 4 x 4 x 704

In [6]:
import os, sys

def get_pelee_net(input_shape=(224,224,3), include_top=True, use_stem_block=True, num_classes=10):
    """
    PeleeNet と呼ばれるDenseNetベースの軽量化なモデル
    係数が2.8Mと少ないながら、ImageNetでMobileNetV2以上の精度を出す
    ImageNetでPeleeNetがTop1:72.6%、MobileNetV2がTop1:72.0%
    StemBlock-DenseBlock-Transition Layer の構造
        StemBlock:入力のダウンサンプリング（入力層のサイズを小さくする）用のブロック
        DenseBlock:DenseNetの「知識の積み重ね、集約」といったConcatで少しずつチャンネル数が増えていく構造のブロック
        Transition Layer:全体に1x1Convを掛けてダウンサンプリングするレイヤー
    Stem blockの有無は、入力解像度・出力解像度が変わる
    Stem blockありが本来のPeleeNetの構造。なしの場合は、いきなりDenseLayerに入る
    
    https://qiita.com/koshian2/items/187e240f478504079e7a
    
    imagenetの重みファイルはないので、h5pyファイル保存せず毎回アーキテクチャ作る
    """
    if include_top == True:
        model = pelee_net_keras.PeleeNet(input_shape=input_shape
                                         , use_stem_block=use_stem_block
                                         , n_classes=num_classes)
    else:
        model = pelee_net_keras.PeleeNet(input_shape=input_shape
                                         , use_stem_block=use_stem_block
                                         , include_top=include_top)
    return model
    

In [7]:
use_stem_block = True

input_shape = (224,224,3) if use_stem_block else (32,32,3)
model = get_pelee_net(input_shape=input_shape, use_stem_block=use_stem_block, num_classes=10)
model.compile(keras.optimizers.SGD(0.4, 0.9), "categorical_crossentropy", ["acc"])

model.summary()

keras.utils.plot_model(model, to_file='PeleeNet_stem_block_True.svg', show_shapes=True)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 112, 112, 32) 896         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 112, 112, 32) 128         conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 112, 112, 32) 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

In [9]:
use_stem_block = False

input_shape = (224,224,3) if use_stem_block else (32,32,3)
model = get_pelee_net(input_shape=input_shape, use_stem_block=use_stem_block, num_classes=10)
model.compile(keras.optimizers.SGD(0.4, 0.9), "categorical_crossentropy", ["acc"])

model.summary()

keras.utils.plot_model(model, to_file='PeleeNet_stem_block_False.svg', show_shapes=True)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_116 (Conv2D)             (None, 32, 32, 16)   64          input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_116 (BatchN (None, 32, 32, 16)   64          conv2d_116[0][0]                 
__________________________________________________________________________________________________
activation_116 (Activation)     (None, 32, 32, 16)   0           batch_normalization_116[0][0]    
__________________________________________________________________________________________________
conv2d_114

## CIFAR10でtest

In [17]:
import tensorflow as tf
import keras
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
from PIL import Image
import pickle
import os

def generator(X, y, batch_size, use_augmentation, shuffle, scale):
    if use_augmentation:
        base_gen = ImageDataGenerator(
            horizontal_flip=True,
            width_shift_range=4.0/32.0,
            height_shift_range=4.0/32.0)
    else:
        base_gen = keras.preprocessing.image.ImageDataGenerator()
    for X_base, y_base in base_gen.flow(X, y, batch_size=batch_size, shuffle=shuffle):
        if scale != 1:
            X_batch = np.zeros((X_base.shape[0], X_base.shape[1]*scale,
                                X_base.shape[2]*scale, X_base.shape[3]), np.float32)
            for i in range(X_base.shape[0]):
                with Image.fromarray(X_base[i].astype(np.uint8)) as img:
                    img = img.resize((X_base.shape[1]*scale, X_base.shape[2]*scale), Image.LANCZOS)
                    X_batch[i] = np.asarray(img, np.float32) / 255.0
        else:
            X_batch = X_base / 255.0
        yield X_batch, y_base
        
def lr_scheduler(epoch):
    x = 0.4
    if epoch >= 70: x /= 5.0
    if epoch >= 120: x /= 5.0
    if epoch >= 170: x /= 5.0
    return x

def train(use_augmentation, use_stem_block):
    tf.logging.set_verbosity(tf.logging.FATAL)
    (X_train, y_train), (X_test, y_test) = keras.datasets.cifar10.load_data()
    y_train = keras.utils.to_categorical(y_train)
    y_test = keras.utils.to_categorical(y_test)

    # generator
    batch_size = 512//8
    scale = 7 if use_stem_block else 1
    train_gen = generator(X_train, y_train, batch_size=batch_size,
                          use_augmentation=use_augmentation, shuffle=True, scale=scale)
    test_gen = generator(X_test, y_test, batch_size=1000//10,
                         use_augmentation=False, shuffle=False, scale=scale)
    
    input_shape = (224,224,3) if use_stem_block else (32,32,3)
    model = get_pelee_net(input_shape=input_shape, use_stem_block=use_stem_block, num_classes=10)
    model.compile(keras.optimizers.SGD(0.4, 0.9), "categorical_crossentropy", ["acc"])
    
    scheduler = keras.callbacks.LearningRateScheduler(lr_scheduler)
    model.fit_generator(train_gen, steps_per_epoch=X_train.shape[0]//batch_size,
                        validation_data=test_gen, validation_steps=X_test.shape[0]//1000,
                        callbacks=[scheduler], epochs=1, max_queue_size=1)
    
if __name__ == "__main__":
    train(True, True)

Epoch 1/1
781/781 [==============================] - 255s 326ms/step - loss: 5.1493 - acc: 0.2292 - val_loss: 2.7251 - val_acc: 0.1870
